In [6]:
from bs4 import BeautifulSoup
import urllib.request

html_doc = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(html_doc,'html.parser')